# VectorStore/QA, MMR support

## Colab-specific setup

Make sure you have a Database and get ready to upload the Secure Connect Bundle and supply the Token string
(see [Pre-requisites](https://cassio.org/start_here/#vector-database) on cassio.org for details).

Likewise, ensure you have the necessary secret for the LLM provider of your choice: you'll be asked to input it shortly
(see [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for details).

_Note: this notebook is part of the CassIO documentation. Visit [this page on cassIO.org](https://cassio.org/frameworks/langchain/qa-maximal-marginal-relevance/)._


In [ ]:
# install required dependencies
! pip install \
    "git+https://github.com/hemidactylus/langchain@cassio-full-preview#egg=langchain" \
    "cassandra-driver>=3.28.0" \
    "cassio==0.0.6" \
    "google-cloud-aiplatform>=1.25.0" \
    "jupyter>=1.0.0" \
    "openai==0.27.7" \
    "python-dotenv==1.0.0" \
    "tensorflow-cpu==2.12.0" \
    "tiktoken==0.4.0" \
    "transformers>=4.29.2"

You will likely be asked to "Restart the Runtime" at this time, as some dependencies
have been upgraded. **Please do restart the runtime now** for a smoother execution from this point onward.

In [ ]:
# Input your database keyspace name:
ASTRA_DB_KEYSPACE = input('Your Astra DB Keyspace name: ')

In [ ]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
ASTRA_DB_TOKEN_BASED_PASSWORD = input('Your Astra DB Token: ')

### Astra DB Secure Connect Bundle

Please upload the Secure Connect Bundle zipfile to connect to your Astra DB instance.

The Secure Connect Bundle is needed to establish a secure connection to the database.
Click [here](https://awesome-astra.github.io/docs/pages/astra/download-scb/#c-procedure) for instructions on how to download it from Astra DB.

In [ ]:
# Upload your Secure Connect Bundle zipfile:
import os
from google.colab import files


print('Please upload your Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
    )

In [ ]:
# colab-specific override of helper functions
from cassandra.cluster import (
    Cluster,
)
from cassandra.auth import PlainTextAuthProvider

# The "username" is the literal string 'token' for this connection mode:
ASTRA_DB_TOKEN_BASED_USERNAME = 'token'


def getCQLSession(mode='astra_db'):
    if mode == 'astra_db':
        cluster = Cluster(
            cloud={
                "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
            },
            auth_provider=PlainTextAuthProvider(
                ASTRA_DB_TOKEN_BASED_USERNAME,
                ASTRA_DB_TOKEN_BASED_PASSWORD,
            ),
        )
        astraSession = cluster.connect()
        return astraSession
    else:
        raise ValueError('Unsupported CQL Session mode')

def getCQLKeyspace(mode='astra_db'):
    if mode == 'astra_db':
        return ASTRA_DB_KEYSPACE
    else:
        raise ValueError('Unsupported CQL Session mode')

### LLM Provider

In the cell below you can choose between **GCP VertexAI** or **OpenAI** for your LLM services.
(See [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for more details).

Make sure you set the `llmProvider` variable and supply the corresponding access secrets in the following cell.

In [ ]:
# Set your secret(s) for LLM access:
llmProvider = 'OpenAI'  # 'GCP_VertexAI'


In [ ]:
if llmProvider == 'OpenAI':
    apiSecret = input(f'Your secret for LLM provider "{llmProvider}": ')
    os.environ['OPENAI_API_KEY'] = apiSecret
elif llmProvider == 'GCP_VertexAI':
    # we need a json file
    print(f'Please upload your Service Account JSON for the LLM provider "{llmProvider}":')
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        vertexAIJsonFileTitle = list(uploaded.keys())[0]
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.getcwd(), vertexAIJsonFileTitle)
    else:
        raise ValueError(
            'No file uploaded. Please re-run the cell.'
        )
else:
    raise ValueError('Unknown/unsupported LLM Provider')

### Colab preamble completed

The following cells constitute the demo notebook proper.

# VectorStore/QA, MMR support

_**NOTE:** this uses Cassandra's "Vector Search" capability.
Make sure you are connecting to a vector-enabled database for this demo._

Cassandra's `VectorStore` allows for Vector Search with the **Maximal Marginal Relevance (MMR)** algorithm.

This is a search criterion that instead of just selecting the _k_ stored documents most relevant to the provided query, first identifies a larger pool of relevant results, and then singles out _k_ of them so that they carry as diverse information between them as possible.

In this way, when the stored text fragments are likely to be redundant, you can optimize token usage and help the models give more comprehensive answers.

_This is very useful, for instance, if you are building a QA chatbot on past Support chat recorded interactions._

First prepare a connection to a vector-search-capable Cassandra and initialize the required LLM and embeddings:

In [1]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.vectorstores import Cassandra

In [2]:
# creation of the DB connection
cqlMode = 'astra_db'
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

Below is the logic to instantiate the LLM and embeddings of choice. We choose to leave it in the notebooks for clarity.

In [3]:
# creation of the LLM resources


if llmProvider == 'GCP_VertexAI':
    from langchain.llms import VertexAI
    from langchain.embeddings import VertexAIEmbeddings
    llm = VertexAI(temperature=0)
    myEmbedding = VertexAIEmbeddings()
    print('LLM+embeddings from VertexAI')
elif llmProvider == 'OpenAI':
    from langchain.llms import OpenAI
    from langchain.embeddings import OpenAIEmbeddings
    llm = OpenAI(temperature=0)
    myEmbedding = OpenAIEmbeddings()
    print('LLM+embeddings from OpenAI')
else:
    raise ValueError('Unknown LLM provider.')

LLM+embeddings from OpenAI


## Create the store

Create a (Cassandra-backed) `VectorStore` and the corresponding LangChain `VectorStoreIndexWrapper`

In [4]:
myCassandraVStore = Cassandra(
    embedding=myEmbedding,
    session=session,
    keyspace=keyspace,
    table_name='vs_test2_' + llmProvider,
)
index = VectorStoreIndexWrapper(vectorstore=myCassandraVStore)

This command simply resets the store in case you want to run this demo repeatedly:

In [5]:
myCassandraVStore.clear()

## Populate the index


Notice that the first four sentences express the same concept, while the **fifth** adds a new detail:

In [6]:
BASE_SENTENCE_0 =     ('The frogs and the toads were meeting in the night '
                       'for a party under the moon.')

BASE_SENTENCE_1 =     ('There was a party under the moon, that all toads, '
                       'with the frogs, decided to throw that night.')

BASE_SENTENCE_2 =     ('And the frogs and the toads said: "Let us have a party '
                       'tonight, as the moon is shining".')

BASE_SENTENCE_3 =     ('I remember that night... toads, along with frogs, '
                       'were all busy planning a moonlit celebration.')

DIFFERENT_SENTENCE =  ('For the party, frogs and toads set a rule: '
                       'everyone was to wear a purple hat.')

Insert the three into the index, specifying "sources" while you're at it (it will be useful later):

In [7]:
myCassandraVStore.add_texts(
    [
        BASE_SENTENCE_0,
        BASE_SENTENCE_1,
        BASE_SENTENCE_2,
        BASE_SENTENCE_3,
        DIFFERENT_SENTENCE,
    ],
    metadatas=[
        {'source': 'Barney\'s story at the pub'},
        {'source': 'Barney\'s story at the pub'},
        {'source': 'Barney\'s story at the pub'},
        {'source': 'Barney\'s story at the pub'},
        {'source': 'The chronicles at the village library'},
    ],
)

['d86ebe8bf2f6fa27ff01db7e3c4a21ab',
 'be4fffdf596f08c1d3f4d9effc2f327e',
 'd25517400eac2ff0eb4c9c2b38d5e7db',
 '7bb2aec568c5577c107a403f2fb1a64e',
 '4654a61925e397ea5f097019b2fa56d2']

## Query the store

Here is the question you'll use to query the index:

In [8]:
QUESTION = 'Tell me about the party that night.'

### Query with "similarity" search type

If you ask for two matches, you will get the two documents most related to the question. But in this case this is something of a waste of tokens:

In [9]:
matchesSim = myCassandraVStore.search(QUESTION, search_type='similarity', k=2)
for i, doc in enumerate(matchesSim):
    print(f'[{i:2}]: "{doc.page_content}"')

[ 0]: "There was a party under the moon, that all toads, with the frogs, decided to throw that night."
[ 1]: "I remember that night... toads, along with frogs, were all busy planning a moonlit celebration."


### Query with MMR

Now, here's what happens with the MMR search type.

_(Not shown here: you can tune the size of the results pool for the first step of the algorithm.)_

In [10]:
matchesMMR = myCassandraVStore.search(QUESTION, search_type='mmr', k=2)
for i, doc in enumerate(matchesMMR):
    print(f'[{i:2}]: "{doc.page_content}"')

[ 0]: "There was a party under the moon, that all toads, with the frogs, decided to throw that night."
[ 1]: "For the party, frogs and toads set a rule: everyone was to wear a purple hat."


## Query the index

Currently, LangChain's higher "index" abstraction does not allow to specify the search type, nor the number of matches subsequently used in creating the answer. So, by running this command you get an answer, all right.

In [11]:
# (implicitly) by similarity
print(index.query(QUESTION, llm=llm))

 The frogs and toads were having a party under the moon that night. They were busy planning and celebrating together.


You can request the question-answering process to provide references (as long as you annotated all input documents with a `source` metadata field):

In [12]:
responseSrc = index.query_with_sources(QUESTION, llm=llm)
print('Automatic chain (implicitly by similarity):')
print(f'  ANSWER : {responseSrc["answer"].strip()}')
print(f'  SOURCES: {responseSrc["sources"].strip()}')

Automatic chain (implicitly by similarity):
  ANSWER : The frogs and toads were planning a party under the moon that night.
  SOURCES: Barney's story at the pub


Here the default is to fetch _four_ documents ... so that the only other text actually carrying additional information is left out!

### The QA Process behind the scenes

In order to exploit the MMR search in end-to-end question-answering pipelines, you need to recreate and manually tweak the steps behind the `query` or `query_with_sources` methods. This takes just a few lines.

First you need a few additional modules:

In [13]:
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.chains.qa_with_sources.retrieval import RetrievalQAWithSourcesChain

You are ready to run two QA chains, identical in all respects (especially in the number of results to fetch, two), except the `search_type`:

#### Similarity-based QA

In [14]:
# manual creation of the "retriever" with the 'similarity' search type
retrieverSim = myCassandraVStore.as_retriever(
    search_type='similarity',
    search_kwargs={
        'k': 2,
        # ...
    },
)
# Create a "RetrievalQA" chain
chainSim = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retrieverSim,
)
# Run it and print results
responseSim = chainSim.run(QUESTION)
print(responseSim)

 The party was held under the moon and was planned by both toads and frogs.


#### MMR-based QA

In [15]:
# manual creation of the "retriever" with the 'MMR' search type
retrieverMMR = myCassandraVStore.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k': 2,
        # ...
    },
)
# Create a "RetrievalQA" chain
chainMMR = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retrieverMMR
)
# Run it and print results
responseMMR = chainMMR.run(QUESTION)
print(responseMMR)

 The party was held under the moon and was attended by both frogs and toads. Everyone was required to wear a purple hat.


#### Answers with sources

You can run the variant of these chains that also returns the source for the documents used in preparing the answer, which makes it even more obvious:

In [16]:
chainSimSrc = RetrievalQAWithSourcesChain.from_chain_type(
    llm,
    retriever=retrieverSim,
)
#
responseSimSrc = chainSimSrc({chainSimSrc.question_key: QUESTION})
print('Similarity-based chain:')
print(f'  ANSWER : {responseSimSrc["answer"].strip()}')
print(f'  SOURCES: {responseSimSrc["sources"].strip()}')

Similarity-based chain:
  ANSWER : The toads and frogs were planning a moonlit celebration.
  SOURCES: Barney's story at the pub


In [17]:
chainMMRSrc = RetrievalQAWithSourcesChain.from_chain_type(
    llm,
    retriever=retrieverMMR,
)
#
responseMMRSrc = chainMMRSrc({chainMMRSrc.question_key: QUESTION})
print('MMR-based chain:')
print(f'  ANSWER : {responseMMRSrc["answer"].strip()}')
print(f'  SOURCES: {responseMMRSrc["sources"].strip()}')

MMR-based chain:
  ANSWER : The party that night was thrown by frogs and toads, and everyone was required to wear a purple hat.
  SOURCES: Barney's story at the pub, The chronicles at the village library


## What now?

This demo is hosted [here](https://cassio.org/frameworks/langchain/qa-maximal-marginal-relevance/) at cassio.org.

Discover the other ways you can integrate 
Cassandra/Astra DB with your ML/GenAI needs,
right **within [your favorite framework](https://cassio.org/frameworks/langchain/about/)**.